In [1]:
!conda install -y -c quantopian ta-lib

Fetching package metadata ...............
Solving package specifications: .

Package plan for installation in environment /home/nbuser/anaconda3_420:

The following NEW packages will be INSTALLED:

    readline:  7.0-ha6073c6_4              
    ta-lib:    0.4.9-np111py35_0 quantopian

The following packages will be UPDATED:

    conda:     4.3.31-py35_0                --> 4.5.11-py35_0       
    conda-env: 2.6.0-h36134e3_1             --> 2.6.0-1             
    pycosat:   0.6.1-py35_1                 --> 0.6.3-py35h6b6bb97_0

conda-env-2.6. 100% |################################| Time: 0:00:00 551.99 kB/s
readline-7.0-h 100% |################################| Time: 0:00:00   7.26 MB/s
pycosat-0.6.3- 100% |################################| Time: 0:00:00   7.21 MB/s
ta-lib-0.4.9-n 100% |################################| Time: 0:00:00   4.67 MB/s
conda-4.5.11-p 100% |################################| Time: 0:00:00   8.58 MB/s


In [4]:
import kiteconnect

## <span style="color:grey">Initialization</span>

In [2]:
%run "KiteConnect_AlgoBase.ipynb" 

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '
:0: UserWarning:

You do not have a working installation of the service_identity module: 'cannot import name 'opentype''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.



##  <span style="color:green">Trading Strategy </span>

In [6]:
class algoTrade(algoTrade_base):
   
    def __init__(s, symbol):
        logger.debug("AlgoTrade Called: "+symbol)
        super(algoTrade, s).__init__(symbol)
        
    
    # Long Strategies
    def long_indicators(s):
        temp_df = 0
        temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
        temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
        temp_df = temp_df | (REF(s.macd,1) <=0)&(s.macd>0)
        temp_df = temp_df | CROSSOVER(s.macd, s.macdsignal)
        temp_df = temp_df | (REF(s.rsi,1) <=50)
        #temp_df = temp_df | (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)
        #temp_df = temp_df & CROSSOVER(s.BBB, s.CLOSE)
        
        return pd.DataFrame(  temp_df, columns=["buy"] )
     
    # Short Strategies
    def short_indicators(s):
        temp_df = 0
        temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80)
        temp_df = temp_df | CROSSOVER(s.fastd, s.fastk)
        temp_df = temp_df | (REF(s.macd,1) >=0)&(s.macd<0)
        temp_df = temp_df | CROSSOVER(s.macdsignal, s.macd)
        temp_df = temp_df | (REF(s.rsi,1) >=50)
        #temp_df = temp_df & CROSSOVER(s.CLOSE, s.BBT)
        return pd.DataFrame( temp_df, columns=["sell"])
    

    
    def long_breakout(s):
        temp_df = s.haCLOSE >= s.BBT.shift(1)
        temp_df = temp_df | (s.haCLOSE >= s.haOPEN.shift(2))
        temp_df = temp_df & ( CROSSOVER(s.OPEN, s.BBT) | CROSSOVER(s.OPEN, s.BBB) | CROSSOVER(s.OPEN, s.BBM))
        return pd.DataFrame( temp_df , columns=["buy"])
    
    def short_breakout(s):
        temp_df = s.haOPEN <= s.BBB.shift(1)
        temp_df = temp_df | (s.haCLOSE <= s.haOPEN.shift(2))
        temp_df = temp_df & ( CROSSOVER(s.BBT, s.OPEN) | CROSSOVER(s.BBB, s.OPEN) | CROSSOVER(s.BBM, s.OPEN))
        return pd.DataFrame( temp_df , columns=["sell"])
    
    def long_ha(s):
        temp_df = (REF(s.haCLOSE,2) < REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) < REF(s.haOPEN,1)) & (s.haCLOSE > s.haOPEN) 
        #temp_df = temp_df & (s.rsi < 40 )
        return pd.DataFrame( temp_df , columns=["buy"])
    
    def short_ha(s):
        temp_df = (REF(s.haCLOSE,2) > REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) > REF(s.haOPEN,1)) & (s.haCLOSE < s.haOPEN)
        #temp_df = temp_df & (s.rsi > 60 )
        return pd.DataFrame( temp_df , columns=["sell"])

    def tradeDecision(s, price_ohlc_df):
        global blackoutEnabled

        super(algoTrade, s).tradeDecision(price_ohlc_df)
        
        #s.rsi = RSI(s.CLOSE, timeperiod=20)
        #s.min = MIN(s.CLOSE, timeperiod=30)
        #s.max = MAX(s.CLOSE, timeperiod=30)
        
        s.buy  = s.long_ha() & s.long_indicators()
        s.sell = s.short_ha() & s.short_indicators()
        
        s.postprocess()
        
        return (s.buy,s.sell)
        
if mode == "Algo":
    x = 'YESBANK'
    gc.collect()
    days=1
    portfolio = pd.DataFrame([x])
    toDate = dt.datetime(2019,5,3,15,15)
    #portfolio = pd.DataFrame(downloadlist)    
    print("\nPortfolio return: "+ '%.2f'%trade_simulator(portfolio, toDate, cd=False, plot=True)+"%")
    
    

UnboundLocalError: local variable 'temp_data' referenced before assignment

In [ ]:
# Backtesting for no of days

if mode == "Algo":
    x = 'YESBANK'
    gc.collect()
    days=2
    portfolio = pd.DataFrame([x])
    
    noofdays = 180
    for i in np.linspace(noofdays,1,noofdays):
        toDate = dt.datetime(2019,5,2,15,15) - dt.timedelta(days=i)
        #print(toDate)
        if isholiday(toDate) == True:
            continue
        print(toDate)
        #portfolio = pd.DataFrame(downloadlist)    
        trade_simulator(portfolio, toDate, cd=False, plot=False)
    
    

In [ ]:
if mode == "Algo":
    x = 'RELIANCE'
    gc.collect()
    days =2
    portfolio = pd.DataFrame([x])
    toDate = dt.datetime(2019,4,30,15,15)
    #portfolio = pd.DataFrame(downloadlist)    
    print("\nPortfolio return: "+ '%.2f'%trade_simulator(portfolio, toDate, cd=True, plot=True)+"%")

In [ ]:
if mode == "Algo":
    x = 'INFRATEL'
    gc.collect()
    portfolio = pd.DataFrame([x])
    toDate = dt.datetime(2019,4,30,15,15)
    #portfolio = pd.DataFrame(downloadlist)    
    print("\nPortfolio return: "+ '%.2f'%trade_simulator(portfolio, toDate, mode="simulator")+"%")

In [1]:
import pandas as pd

In [2]:
!ls -l --block-size=MB kite_data

total 0MB
-rw-r--r-- 1 nbuser nbuser   1MB May 21 09:17 access_token.txt
-rw-r--r-- 1 nbuser nbuser 132MB May 21 15:44 kite_cache.h5


In [7]:
pd.HDFStore('kite_data/kite_cache.h5').get('/minute/NSE/YESBANK').head()

,close,high,low,open,volume
date,,,,,
2019-04-22 09:15:00,250.80,253.95,249.40,253.95,577837
2019-04-22 09:16:00,247.30,251.60,247.30,251.50,466210
2019-04-22 09:17:00,249.30,249.60,247.45,247.45,257220
2019-04-22 09:18:00,249.20,249.40,248.00,249.35,343875
2019-04-22 09:19:00,247.95,249.20,247.95,249.10,297897
